This brief example shows how to use the isochrones package to extract an isochrone for a specific location and time. 

It assumes that 
* the isochrones package has been properly installed and additional dependencies (matplotlib, numpy) have been installed (e.g. with `pip install isochrones[example]`)
* OpenTripPlanner (OTP) is running locally on the 8080 port
* the OTP data is stored in the `otp/graphs/main` subfolder

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import colors as colors
import datetime
import numpy as np
import geopandas as gpd
import contextily as ctx


from isochrones.isochrones import calculate_isochrones, intersect_isochrones

In [ ]:
lat = 46.20730
lon = 6.12686
cutoffSec = [1800, 3600, 5400]
date_time = datetime.datetime(2022, 5, 20, 16, 0)
ssl = False
hostname = "localhost"
port = 8080
router = "ara_geneve_graphe_diff_vitesse"

# Create a normalized colormap based on cutoffSec values
norm = colors.Normalize(vmin=min(cutoffSec), vmax=max(cutoffSec))
cmap = plt.cm.viridis

In [ ]:
output = calculate_isochrones(
    lat, lon, cutoffSec, date_time, ssl, hostname, port, router
)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

output.plot(
    ax=ax, column="time", alpha=0.5, legend=True, cmap="viridis", facecolor="none"
)

ctx.add_basemap(
    ax,
    source=ctx.providers.OpenStreetMap.CH,
    attribution="© OpenStreetMap contributors",
    zoom=12,
)

In [ ]:
def plot_with_basemap(gdf, random_points=None, figsize=(15, 15), zoom=12):
    # Convert to Web Mercator for contextily
    gdf_web = gdf.to_crs(epsg=3857)

    # Create figure
    fig, ax = plt.subplots(figsize=figsize)

    # Plot isochrones
    gdf_web.plot(ax=ax, column="time", alpha=0.5, legend=True, cmap="viridis")

    # Plot random points if provided
    if random_points is not None:
        random_points_web = random_points.to_crs(epsg=3857)
        random_points_web.plot(ax=ax, color="red", markersize=5)

    ctx.add_basemap(
        ax,
        crs=gdf_web.crs.to_string(),
        source=ctx.providers.SwissFederalGeoportal.NationalMapColor,
        attribution="© OpenStreetMap contributors",
        zoom=zoom,
    )

    # Remove axes
    # ax.set_axis_off()

    return fig, ax

In [ ]:
# Create plot with base layer
fig, ax = plot_with_basemap(output)
plt.show()

In [ ]:
# Randomly sample points within the bounding box of the isochrones
# Set the seed
np.random.seed(81)

n_points = 200
minx, miny, maxx, maxy = output.total_bounds
random_points = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(
        np.random.uniform(minx, maxx, n_points), np.random.uniform(miny, maxy, n_points)
    )
)

fig, ax = plt.subplots(figsize=(10, 10))
output.plot(ax=ax, column="time", alpha=0.5, legend=True, cmap=cmap)
random_points.plot(ax=ax, color="red", markersize=5)

iso = plt.show()

In [ ]:
# Intersect the random points with each isochrones stored in the output
intersected_random = intersect_isochrones(output, random_points)

# Plot the results, with each isochrone in a separate subplot
fig, axes = plt.subplots(ncols=len(cutoffSec), figsize=(15, 10))

for ax, i in zip(axes, range(len(cutoffSec))):
    output_data = output[output["time"] == cutoffSec[i]]
    intersected_data = intersected_random[intersected_random["time"] == cutoffSec[i]]

    color = cmap(norm(cutoffSec[i]))
    output_data.plot(ax=ax, color=color, alpha=0.5, legend=True)

    if not intersected_data.empty:
        intersected_data.plot(ax=axes[i], color="red", markersize=5)

    # Set consistent bounds for each subplot
    ax.set_xlim(minx, maxx)
    ax.set_ylim(miny, maxy)

    # Add title to show the time cutoff
    ax.set_title(f"{cutoffSec[i] / 60:.0f} minutes")

plt.tight_layout()
plt.show()